In [82]:
import os
import io
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, ParameterGrid
import pickle
from itertools import chain
import requests

import math

pickle_dir = 'adult_pickles'

# helper function for data frame str / summary
def rstr(df):
    return df.shape, df.apply(lambda x: [x.unique()])

# helper function for pickling files
def pickle_path(filename):
    return(pickle_dir + '\\' + filename)

url="https://raw.githubusercontent.com/propublica/compas-analysis/master/compas-scores-two-years.csv"
s=requests.get(url).content
compas = pd.read_csv(io.StringIO(s.decode('utf-8')))

var_names = compas.columns.tolist()

review_vars = ['age', 'c_charge_degree', 'race', 'age_cat', 'score_text', 'sex', 'priors_count', 
'days_b_screening_arrest', 'decile_score', 'is_recid', 'two_year_recid', 'c_jail_in', 'c_jail_out']

extra_vars = ['compas_screening_date', 'juv_fel_count', 'juv_misd_count'
, 'juv_other_count', 'c_offense_date', 'c_days_from_compas'
, 'c_charge_desc']

exclude_vars = [v for v in var_names if v not in review_vars] # and v not in extra_vars]

compas = compas.drop(exclude_vars, axis=1)

vars_types = ['nominal'
           , 'continuous'
           , 'nominal'
           , 'nominal'
           , 'continuous'
           , 'continuous'
           , 'continuous'
           , 'continuous'
           , 'continuous'
           , 'nominal'
           , 'nominal'
           , 'nominal'              
           , 'nominal']

class_col = 'is_recid'

compas = compas[~pd.isnull(compas.days_b_screening_arrest.values)]
compas = compas[compas.days_b_screening_arrest.values <= 30]
compas = compas[compas.days_b_screening_arrest.values >= -30]
compas = compas[compas.is_recid != -1]
compas = compas[compas.c_charge_degree != 'O']

parse_date = lambda x : datetime.datetime.strptime(x, '%Y-%m-%d %I:%M:%S') 

compas.c_jail_in = compas.c_jail_in.map(parse_date)
compas.c_jail_out = compas.c_jail_out.map(parse_date)
compas['length_of_stay'] = compas.c_jail_out - compas.c_jail_in

In [83]:
rstr(compas)

((6172, 14),
 sex                                                         [[Male, Female]]
 age                        [[69, 34, 24, 44, 41, 43, 39, 27, 23, 37, 47, ...
 age_cat                           [[Greater than 45, 25 - 45, Less than 25]]
 race                       [[Other, African-American, Caucasian, Hispanic...
 decile_score                               [[1, 3, 4, 6, 10, 5, 9, 2, 7, 8]]
 priors_count               [[0, 4, 14, 3, 1, 7, 6, 5, 13, 8, 9, 21, 2, 15...
 days_b_screening_arrest    [[-1.0, 0.0, -20.0, 22.0, -2.0, -24.0, -13.0, ...
 c_jail_in                  [[2013-08-13 06:03:42, 2013-01-26 03:45:27, 20...
 c_jail_out                 [[2013-08-14 05:41:20, 2013-02-05 05:36:53, 20...
 c_charge_degree                                                     [[F, M]]
 is_recid                                                            [[0, 1]]
 score_text                                             [[Low, Medium, High]]
 two_year_recid                                    

In [ ]:
compas = compas.drop(['id', 'name', 'first', 'last', 'dob'
                      , 'c_case_number', 'r_case_number'
                      , 'violent_recid', 'vr_case_number'
                     , 'type_of_assessment', 'v_type_of_assessment'
                     , 'priors_count.1'], axis=1)



rstr(compas)